# Positive Tone Data Generator

This notebook contains code for an MLP neural network that generates positively toned data based on the dataset.

### Import Libraries

In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import csv
import pandas as pd
from gensim.models import Word2Vec
from nltk import regexp_tokenize

### Input

In [2]:
df_raw = pd.read_csv('poem_sentiment.csv', header=None, index_col=0, names=['Text', 'Sentiment'])
df_raw

,Text,Sentiment
0,with pale blue berries. in these peaceful shad...,1.0
1,it flows so long as falls the rain,0.0
2,"and that is why, the lonesome day",-1.0
3,"when i peruse the conquered fame of heroes, an...",2.0
4,of inward strife for truth and liberty.,2.0
...,...,...
887,to his ears there came a murmur of far seas be...,0.0
888,"the one good man in the world who knows me, --",1.0
889,faint voices lifted shrill with pain,-1.0
890,"an', fust you knowed on, back come charles the...",0.0


In [3]:
positive = df_raw[df_raw['Sentiment'] > 0]
positive

,Text,Sentiment
0,with pale blue berries. in these peaceful shad...,1.0
3,"when i peruse the conquered fame of heroes, an...",2.0
4,of inward strife for truth and liberty.,2.0
5,the red sword sealed their vows!,2.0
16,that has a charmingly bourbon air.,1.0
...,...,...
870,their first-born brother as a god.,1.0
876,and so i should be loved and mourned to-night.,2.0
877,"and _channing_, with his bland, superior look",2.0
884,"how your soft opera-music changed, and the dru...",2.0


# Generate positively toned data

In [4]:
raw_text = positive['Text'].to_string(index=False)
raw_text[:1000]

'with pale blue berries. in these peaceful shades--\nwhen i peruse the conquered fame of heroes, and...\n           of inward strife for truth and liberty.\n                  the red sword sealed their vows!\n                that has a charmingly bourbon air.\n          brightly expressive as the twins of leda\n               in monumental pomp! no grecian drop\n                    the hostile cohorts melt away;\nand lips where heavenly smiles would hang and b...\n                         honour to the bugle-horn!\n                       if the pure and holy angels\n        upon the thought of perfect noon. and when\n      thy hands all cunning arts that women prize.\n             reasoning to admiration, and with mee\n           it shines superior on a throne of gold:\n    take the warm welcome of new friends with thee\n                  augmented, sweet, a hundred fold\n                every day a rich reward will give;\n                                 gay little heart!\n         am

In [5]:
# Remove all non-ASCII characters
processed_text = re.sub(r'[^\x00-\x7f]', r'', raw_text).lower()
processed_text[:1000]

'with pale blue berries. in these peaceful shades--\nwhen i peruse the conquered fame of heroes, and...\n           of inward strife for truth and liberty.\n                  the red sword sealed their vows!\n                that has a charmingly bourbon air.\n          brightly expressive as the twins of leda\n               in monumental pomp! no grecian drop\n                    the hostile cohorts melt away;\nand lips where heavenly smiles would hang and b...\n                         honour to the bugle-horn!\n                       if the pure and holy angels\n        upon the thought of perfect noon. and when\n      thy hands all cunning arts that women prize.\n             reasoning to admiration, and with mee\n           it shines superior on a throne of gold:\n    take the warm welcome of new friends with thee\n                  augmented, sweet, a hundred fold\n                every day a rich reward will give;\n                                 gay little heart!\n         am

In [47]:
# Get word tokens from text
word_tokens = regexp_tokenize(processed_text, pattern=r'[^\S\r]+|[\.,;!?()--_"]', gaps=True)
word_tokens.append('\n')
print(f"Number of word tokens: {len(word_tokens)}")
word_tokens[:10]

Number of word tokens: 1293


['with',
 'pale',
 'blue',
 'berries',
 'in',
 'these',
 'peaceful',
 'shades',
 'when',
 'i']

In [48]:
# Get unique word tokens from word tokens
unique_words = sorted(list(set(word_tokens)))
print(f"Number of unique word tokens: {len(unique_words)}")

Number of unique word tokens: 685


In [49]:
# Create vocabulary of word tokens
word_vocabulary = unique_words
word_vocabulary[:10]

['\n',
 "'tis",
 'a',
 'abloom',
 'abodes',
 'about',
 'accordance',
 'adam',
 'adept',
 'admiration']

In [53]:
# Create index-word mappings 
indices_words = dict((index, word) for index, word in enumerate(unique_words))
indices_words

{0: '\n',
 1: "'tis",
 2: 'a',
 3: 'abloom',
 4: 'abodes',
 5: 'about',
 6: 'accordance',
 7: 'adam',
 8: 'adept',
 9: 'admiration',
 10: 'after',
 11: 'again',
 12: 'ah',
 13: 'air',
 14: 'all',
 15: 'already',
 16: 'am',
 17: 'amidst',
 18: 'among',
 19: 'an',
 20: 'and',
 21: 'angel',
 22: 'angels',
 23: 'angry',
 24: 'are',
 25: 'arms',
 26: 'around',
 27: 'art',
 28: 'arts',
 29: 'as',
 30: 'ascends',
 31: 'ashes',
 32: 'aspiring',
 33: 'assay',
 34: 'augmented',
 35: 'away',
 36: 'awe',
 37: 'ay',
 38: 'b',
 39: 'bare',
 40: 'be',
 41: 'beauteous',
 42: 'beautiful',
 43: 'beauty',
 44: "beauty'",
 45: 'because',
 46: 'before',
 47: 'bells',
 48: 'beneath',
 49: 'bent',
 50: 'berries',
 51: 'best',
 52: 'betrayed',
 53: 'between',
 54: 'blades',
 55: 'bland',
 56: 'blazing',
 57: 'blessed',
 58: 'blessing',
 59: 'blind',
 60: 'blue',
 61: 'bolt',
 62: 'born',
 63: 'borrowing',
 64: 'boston',
 65: 'bourbon',
 66: 'bow',
 67: 'brave',
 68: 'braver',
 69: 'breast',
 70: 'bright',
 71

In [54]:
# Create word-index mappings
word_indices = dict((word, index) for index, word in enumerate(unique_words))
word_indices

{'\n': 0,
 "'tis": 1,
 'a': 2,
 'abloom': 3,
 'abodes': 4,
 'about': 5,
 'accordance': 6,
 'adam': 7,
 'adept': 8,
 'admiration': 9,
 'after': 10,
 'again': 11,
 'ah': 12,
 'air': 13,
 'all': 14,
 'already': 15,
 'am': 16,
 'amidst': 17,
 'among': 18,
 'an': 19,
 'and': 20,
 'angel': 21,
 'angels': 22,
 'angry': 23,
 'are': 24,
 'arms': 25,
 'around': 26,
 'art': 27,
 'arts': 28,
 'as': 29,
 'ascends': 30,
 'ashes': 31,
 'aspiring': 32,
 'assay': 33,
 'augmented': 34,
 'away': 35,
 'awe': 36,
 'ay': 37,
 'b': 38,
 'bare': 39,
 'be': 40,
 'beauteous': 41,
 'beautiful': 42,
 'beauty': 43,
 "beauty'": 44,
 'because': 45,
 'before': 46,
 'bells': 47,
 'beneath': 48,
 'bent': 49,
 'berries': 50,
 'best': 51,
 'betrayed': 52,
 'between': 53,
 'blades': 54,
 'bland': 55,
 'blazing': 56,
 'blessed': 57,
 'blessing': 58,
 'blind': 59,
 'blue': 60,
 'bolt': 61,
 'born': 62,
 'borrowing': 63,
 'boston': 64,
 'bourbon': 65,
 'bow': 66,
 'brave': 67,
 'braver': 68,
 'breast': 69,
 'bright': 70,
 'b

In [76]:
# Create x (input): Split text into blocks, where each block has the same amount of words
# Create y (targets): For each x input, the y is the word that comes next
# The model should learn to predict y from the input x

block_size = 5
step = 1

x = []
y = []

for i in range(0, len(word_tokens) - block_size, step):
    x.append(word_tokens[i: i+block_size])
    y.append(word_tokens[i + block_size])

In [82]:
# Inspect x
x[:5]

[['with', 'pale', 'blue', 'berries', 'in'],
 ['pale', 'blue', 'berries', 'in', 'these'],
 ['blue', 'berries', 'in', 'these', 'peaceful'],
 ['berries', 'in', 'these', 'peaceful', 'shades'],
 ['in', 'these', 'peaceful', 'shades', 'when']]

In [78]:
# Check number of blocks
len(x)

1288

In [79]:
# Create one-hot encoding of x
x_encoded = []

for x_arr in x:
    x_ints = [word_indices[item] for item in x_arr]
    
    x_row = []
    for item in x_ints:
        x_vector = np.zeros(len(unique_words))
        x_vector[item] = 1
        x_row.append(x_vector)
        
    x_encoded.append(x_row)
    
x_encoded = np.array(x_encoded)

In [84]:
# Inspect y
y[:5]

['these', 'peaceful', 'shades', 'when', 'i']

In [85]:
# Convert each word in y into their corresponding indices
y_ints = [word_indices[item] for item in y]
y_ints[:5]

[591, 424, 512, 651, 279]

In [87]:
# Create one-hot encoding of y
y_encoded = []

for item in y_ints:
    y_vector = np.zeros(len(unique_words))
    y_vector[item] = 1
    y_encoded.append(y_vector)

y_encoded = np.array(y_encoded)

In [104]:
class TextGenerator(nn.Module):
    def __init__(self, input_dim, output_dim, block_size):
        super().__init__()

        self.embeddings = nn.Linear(input_dim, 2000)
        self.hidden = nn.Linear(2000, 1200)
        self.output = nn.Linear(1200, output_dim)
        
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.sigmoid(self.embeddings(x))
        x = self.tanh(self.hidden(x))
        x = self.softmax(self.output(x))

        return x

In [105]:
# Get size of input for training the model
input_size = x_encoded[0].ravel().shape[0]
input_size

3425

In [106]:
# Instantiate model
model = TextGenerator(input_size, len(unique_words), block_size)

# Print model configuration
model

TextGenerator(
  (embeddings): Linear(in_features=3425, out_features=2000, bias=True)
  (hidden): Linear(in_features=2000, out_features=1200, bias=True)
  (output): Linear(in_features=1200, out_features=685, bias=True)
  (tanh): Tanh()
  (sigmoid): Sigmoid()
  (softmax): Softmax(dim=1)
)

In [107]:
# Create custom Dataset class
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.n_samples = len(x)
    
    def __getitem__(self, index):
        return self.x[index].ravel(), self.y[index]
    
    def __len__(self):
        return self.n_samples

In [109]:
# Allocate tensors to the device used for computation
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Performing torch operations on {device} device")

# Create x and y PyTorch tensors
x = torch.tensor(x_encoded).float().to(device)
y = torch.tensor(y_encoded).float().to(device)

Performing torch operations on cpu device


In [110]:
# Create training dataset using custom Dataset class
training_ds = CustomDataset(x, y)

In [111]:
# Load training dataset into DataLoader
from torch.utils.data import DataLoader

batch_size = 10

train_loader = DataLoader(
    training_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False
)

In [113]:
# Define model optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.00001)
criterion = nn.CrossEntropyLoss()

In [114]:
# Define function to train model
def train_fn(loader, model, optimizer, loss_fn, device):
    loop = tqdm(loader)

    ave_loss = 0
    count = 0 
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.to(device=device)
        
        # Forward
        predictions = model.forward(data)
        loss = loss_fn(predictions, targets)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update tqdm loading bar
        loop.set_postfix(loss=loss.item())

        count += 1
        ave_loss += loss.item()
    
    ave_loss = ave_loss / count

    return ave_loss

In [115]:
def generate_text(model, seed_text, num_words):
    device = 'cpu'
    model.eval()
    
    # Convert seed_text to input tensor
    seed_encoded = []
    for word in seed_text.split():
        word_index = word_indices[word]
        word_encoded = np.zeros(len(unique_words))
        word_encoded[word_index] = 1
        seed_encoded.append(word_encoded)
    seed_encoded = np.array(seed_encoded)
    seed_encoded = np.expand_dims(seed_encoded, axis=0)
    seed_tensor = torch.tensor(seed_encoded).float().to(device)

    # Generate text
    generated_text = seed_text
    for i in range(num_words):
        predictions = model(seed_tensor)
        predicted_index = torch.argmax(predictions, dim=1).item()
        predicted_word = indices_words[predicted_index]
        generated_text += ' ' + predicted_word
        
        # Update seed tensor with predicted word
        predicted_encoded = np.zeros(len(unique_words))
        predicted_encoded[predicted_index] = 1
        predicted_encoded = np.expand_dims(predicted_encoded, axis=0)
        seed_tensor = torch.cat((seed_tensor[:, 1:, :], torch.tensor(predicted_encoded).float().to(device)), axis=1)

    return generated_text

In [116]:
# Train model
epochs = 2 # TODO: CHANGE TO 300 ON FINAL DATA; CURRENTLY 2 FOR TESTING PURPOSES
average_losses = []

for epoch in range(epochs):
    print("Epoch: {}".format(epoch))
    ave_loss = train_fn(train_loader, model, optimizer, criterion, device)
    
    print("Ave Loss: {}".format(ave_loss))
    average_losses.append(ave_loss)

Epoch: 0


100%|█████████████████████████████████████████████████████████████████████| 129/129 [00:12<00:00, 10.46it/s, loss=6.41]


Ave Loss: 6.496148800665094
Epoch: 1


100%|█████████████████████████████████████████████████████████████████████| 129/129 [00:12<00:00, 10.67it/s, loss=6.41]

Ave Loss: 6.470265998396763


In [118]:
# Generate text sample from model output
word_count = 100
text = []
paragraphs = 5

word1, word2 = "\n", "\n"

for p in range(paragraphs):
    for i in range(word_count):
        phrase = [word1, word2]
        x_ints = [word_indices[item] for item in phrase]
        x_vector = []

        for item in x_ints:
            x_item = np.zeros(len(unique_words))
            x_item[item] = 1
            x_vector.append(x_item)

        initial_input = torch.tensor([np.array([x_vector]).ravel()]).float()

        output = model(initial_input)[0].detach().cpu().numpy()

        # Workaround to fix occasional sum(pvals[:-1]) > 1.0  bug from implicit casting in np.random.multinomial 
        output = output.astype(float)
        output /= output.sum()

        index = np.where(np.random.multinomial(1, output) == 1)[0][0]
        word3 = indices_words[index]
        text.append(word3)

        # Use generated word from this run as seed for next run
        word1, word2 = word2, word3
    
    text.append(r'\n\n')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1370 and 3425x2000)

In [ ]:
print("Generated Text:")
text = ' '.join(text)
print(text)

In [ ]:
with open('positive_generated.csv', 'a') as csvfile:
    fieldnames = ['result']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writerow({'result': 'True'})

# Convert to Feature Vectors

In [3]:
sentences = []

for index, row in positive.iterrows():
    sentences.append(row['Text'].split(' '))
    
sentences

[['with', 'pale', 'blue', 'berries.', 'in', 'these', 'peaceful', 'shades--'],
 ['it', 'flows', 'so', 'long', 'as', 'falls', 'the', 'rain'],
 ['and', 'that', 'is', 'why,', 'the', 'lonesome', 'day'],
 ['when',
  'i',
  'peruse',
  'the',
  'conquered',
  'fame',
  'of',
  'heroes,',
  'and',
  'the',
  'victories',
  'of',
  'mighty',
  'generals,',
  'i',
  'do',
  'not',
  'envy',
  'the',
  'generals'],
 ['of', 'inward', 'strife', 'for', 'truth', 'and', 'liberty.'],
 ['the', 'red', 'sword', 'sealed', 'their', 'vows!'],
 ['and', 'very', 'venus', 'of', 'a', 'pipe.'],
 ['who', 'the', 'man,', 'who,', 'called', 'a', 'brother.'],
 ['and', 'so', 'on.', 'then', 'a', 'worthless', 'gaud', 'or', 'two'],
 ['to', 'hide', 'the', 'orb', 'of', 'truth--and', 'every', 'throne'],
 ['the', "call's", 'more', 'urgent', 'when', 'he', 'journeys', 'slow.'],
 ['with', 'the', '_quart', "d'heure_", 'of', 'rabelais!'],
 ['and',
  'match,',
  'and',
  'bend,',
  'and',
  'thorough-blend,',
  'in',
  'her',
  'colo

In [4]:
vector_size = 100
model = Word2Vec(sentences, vector_size=vector_size)

In [5]:
vocab = model.wv.index_to_key
vocab_length = len(vocab)

print(f'Vocabulary Size: {format(vocab_length)}')

Vocabulary Size: 156


In [6]:
vocab

['the',
 'and',
 'of',
 'to',
 'a',
 'in',
 'i',
 'with',
 'that',
 'his',
 'is',
 'on',
 'as',
 'but',
 'it',
 'he',
 'from',
 'my',
 'all',
 'you',
 'her',
 'for',
 'not',
 'their',
 'when',
 'thy',
 'by',
 'our',
 'she',
 'have',
 'so',
 'they',
 'like',
 'we',
 'are',
 'was',
 'your',
 'what',
 'or',
 'at',
 'who',
 'how',
 'had',
 'were',
 'would',
 'be',
 'its',
 'thou',
 'this',
 'then',
 'which',
 'did',
 'shall',
 'me',
 'eyes',
 'an',
 'yet',
 'upon',
 'if',
 'see',
 'no',
 'will',
 '',
 'down',
 'while',
 'those',
 'old',
 'where',
 'heart',
 'set',
 'some',
 'been',
 'through',
 'though',
 'de',
 'three',
 'him',
 'us',
 'these',
 'little',
 'thee',
 'has',
 'than',
 'never',
 'came',
 'here',
 'up',
 'each',
 'love',
 'far',
 'o',
 'make',
 'said',
 'again',
 'once',
 'there',
 'long',
 'one',
 'whose',
 'still',
 'could',
 'more',
 'out',
 'man',
 'hand',
 "o'er",
 'even',
 'till',
 'now',
 'thus',
 'let',
 'them',
 '--',
 'day',
 'ye',
 'nor',
 'made',
 'can',
 'god',
 '

In [7]:
model.wv.similarity('i', 'me')

0.46391234

In [8]:
vectors = [model.wv[word] for word in vocab]
len(vectors[0])

100

In [9]:
def sentence_to_vector_averaging(sentence, model):
    embeddings = [model.wv[word] for word in sentence if word in vocab]
    
    if len(embeddings) == 0:
        return np.zeros(model.vector_size)
    else:
        return np.mean(embeddings, axis=0)

In [10]:
features = []

for sentence in sentences:
    features.append(sentence_to_vector_averaging(sentence, model))

In [11]:
columns = [f'x{i}' for i in range(model.vector_size)]

df_text = pd.DataFrame(features, columns=columns)
df_text

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99
0,0.000824,0.005484,0.006198,-0.001346,-0.001189,-0.011986,0.008633,0.010692,-0.008599,-0.010502,...,0.010697,0.007396,-0.004039,0.004632,0.014197,0.002761,-0.001273,-0.009714,0.004657,-0.005361
1,-0.001179,0.005808,0.005791,0.003188,0.000606,-0.013482,0.011093,0.017929,-0.005406,-0.005318,...,0.011245,0.004977,0.003930,0.001132,0.015705,0.002386,-0.001763,-0.004498,-0.000880,0.005830
2,-0.003787,0.008056,0.005772,0.007102,-0.001452,-0.016851,0.009702,0.018623,-0.012773,-0.007454,...,0.010073,0.005197,0.003181,0.003394,0.013705,0.006606,0.003267,-0.005551,0.002479,-0.000262
3,-0.001118,0.006276,0.002892,0.006019,-0.000298,-0.015582,0.011757,0.021484,-0.015971,-0.006630,...,0.009934,0.004982,0.001451,0.002435,0.016251,0.004537,0.000746,-0.009897,0.003283,0.002597
4,-0.003520,0.009541,0.004526,0.007233,0.002632,-0.008736,0.004245,0.016388,-0.012425,-0.001741,...,0.010487,0.003139,0.004304,-0.002302,0.010045,0.005573,0.001774,-0.002679,-0.001041,0.001057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,-0.002745,0.005546,0.001635,0.002065,0.000996,-0.010701,0.005857,0.015556,-0.010890,-0.001833,...,0.004850,0.004627,0.003388,0.001956,0.012908,0.001457,-0.001307,-0.005059,0.003821,-0.000298
888,-0.001161,0.000725,0.003928,0.001148,0.000270,-0.013936,0.009739,0.012932,-0.010593,-0.004040,...,0.010232,0.008675,0.000876,0.000442,0.013033,0.007242,-0.000383,-0.007591,0.000694,0.001657
889,0.001803,0.003723,0.001227,0.008650,-0.002200,-0.003292,0.001071,0.006353,-0.010894,-0.000338,...,0.007757,-0.000665,-0.002856,0.006415,0.005263,0.001108,-0.000322,-0.007347,0.000907,-0.006464
890,0.001242,0.009362,0.005818,0.010893,-0.008494,-0.014399,0.016422,0.019538,-0.012709,-0.009564,...,0.010749,0.005814,-0.002229,0.007829,0.013118,0.001911,-0.009071,-0.006203,0.004651,0.008411


In [21]:
df_text['y'] = [float(x) for x in positive['Sentiment']]
df_text

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,y
0,0.000824,0.005484,0.006198,-0.001346,-0.001189,-0.011986,0.008633,0.010692,-0.008599,-0.010502,...,0.007396,-0.004039,0.004632,0.014197,0.002761,-0.001273,-0.009714,0.004657,-0.005361,1.0
1,-0.001179,0.005808,0.005791,0.003188,0.000606,-0.013482,0.011093,0.017929,-0.005406,-0.005318,...,0.004977,0.003930,0.001132,0.015705,0.002386,-0.001763,-0.004498,-0.000880,0.005830,0.0
2,-0.003787,0.008056,0.005772,0.007102,-0.001452,-0.016851,0.009702,0.018623,-0.012773,-0.007454,...,0.005197,0.003181,0.003394,0.013705,0.006606,0.003267,-0.005551,0.002479,-0.000262,-1.0
3,-0.001118,0.006276,0.002892,0.006019,-0.000298,-0.015582,0.011757,0.021484,-0.015971,-0.006630,...,0.004982,0.001451,0.002435,0.016251,0.004537,0.000746,-0.009897,0.003283,0.002597,2.0
4,-0.003520,0.009541,0.004526,0.007233,0.002632,-0.008736,0.004245,0.016388,-0.012425,-0.001741,...,0.003139,0.004304,-0.002302,0.010045,0.005573,0.001774,-0.002679,-0.001041,0.001057,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887,-0.002745,0.005546,0.001635,0.002065,0.000996,-0.010701,0.005857,0.015556,-0.010890,-0.001833,...,0.004627,0.003388,0.001956,0.012908,0.001457,-0.001307,-0.005059,0.003821,-0.000298,0.0
888,-0.001161,0.000725,0.003928,0.001148,0.000270,-0.013936,0.009739,0.012932,-0.010593,-0.004040,...,0.008675,0.000876,0.000442,0.013033,0.007242,-0.000383,-0.007591,0.000694,0.001657,1.0
889,0.001803,0.003723,0.001227,0.008650,-0.002200,-0.003292,0.001071,0.006353,-0.010894,-0.000338,...,-0.000665,-0.002856,0.006415,0.005263,0.001108,-0.000322,-0.007347,0.000907,-0.006464,-1.0
890,0.001242,0.009362,0.005818,0.010893,-0.008494,-0.014399,0.016422,0.019538,-0.012709,-0.009564,...,0.005814,-0.002229,0.007829,0.013118,0.001911,-0.009071,-0.006203,0.004651,0.008411,0.0
